____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Cátedra de Investigación Operativa__\
__Autor: Rodrigo Maranzana__
____

# Ejercicio 7

Los consumidores de café en el área de Pontevedra usan tres marcas A, B, C. En marzo de 2016 se hizo una encuesta en lo que entrevistó a las 8450 personas que compran café y los resultados fueron:

| Desde/Hasta| A | B  | C  |
|--:|--:|---:|---:|
| A |507|845 |338 |
| B |676|2028|676|
| C |845|845 |1690 |

* Si las compras se hacen mensualmente, ¿cuál será la distribución del mercado de café en Pontevedra en el mes de junio?
* A la larga, ¿cómo se distribuirán los clientes de café?
* En junio, cual es la proporción de clientes leales a sus marcas de café?

## Datos Iniciales

Importamos las librerías necesarias.

In [1]:
import numpy as np

Ingresamos el vector de estado inicial:

In [2]:
# Ingreso de vector de estado inicial absulto:
p_0 = np.array([1690, 3380, 3380])

# Relativizamos respecto a la suma de cada fila:
p_0 = p_0 / sum(p_0) # Relativizar

# Printeamos
print(f'Vector de estado inicial: \n{p_0}')

Vector de estado inicial: 
[0.2 0.4 0.4]


Ingresamos los datos de la matriz de transición en una matriz numpy:

In [3]:
# Ingreso de matriz de transición absoluta:
M_abs = np.array([[507, 845, 338], 
                  [676, 2028, 676], 
                  [845, 845, 1690]])


# Relativizamos respecto a la suma de cada fila:
T = np.dot(np.diag(1/np.sum(M_abs, axis=1)), M_abs)

# Printeamos:
print(f'Matriz de transición: \n{T}')

Matriz de transición: 
[[0.3  0.5  0.2 ]
 [0.2  0.6  0.2 ]
 [0.25 0.25 0.5 ]]


In [4]:
M_abs = np.array([[507, 845, 338], [676, 2028, 676], [845, 845, 1690]])
T = np.zeros(M_abs.shape)
for i in range(0, T.shape[0]):
    T[i] = M_abs[i] / sum(M_abs[i])
    
T

array([[0.3 , 0.5 , 0.2 ],
       [0.2 , 0.6 , 0.2 ],
       [0.25, 0.25, 0.5 ]])

## Punto A: Distribución del mercado en junio

En primer lugar, calculamos la matriz de transición habiendo pasado 3 meses desde Marzo: elevamos la matriz al cubo usando el método de la potencia de álgebra lineal de la librería Numpy.

In [5]:
# Primer punto: distribución en el mes de junio (3 meses desde marzo):
# Matriz de transición de 3 meses:
T_3 = np.linalg.matrix_power(T, 3)

# Printeamos la matriz de transicion de 3 pasos:
print(f'Matriz de transición a tiempo 3: \n{T_3}')

Matriz de transición a tiempo 3: 
[[0.237  0.485  0.278 ]
 [0.236  0.486  0.278 ]
 [0.2425 0.4525 0.305 ]]


Calculamos: $ p_0 T^3 = p_3 $

In [6]:
# Estado final a 3 meses:
p_3 = np.dot(p_0, T_3)

# Printeamos el vector de estado final:
print(f'Vector de probabilidades a t=3: \n{p_3}')

Vector de probabilidades a t=3: 
[0.2388 0.4724 0.2888]


## Punto B: Cálculo de estado estable:

### Método con sistema de ecuaciones ya cargadas:
Dado el siguiente sistema para encontrar el estado estable:
$$\pi T = \pi$$
$$ \sum_i{\pi_i} = 1$$

Lo reordenamos y lo escribimos de la siguiente forma:
$$ Ax = b $$

Cargamos la Matriz A:

In [7]:
# Matriz A:
A = np.array([[-0.70, 0.20, 0.25],
              [ 0.50,-0.40, 0.25],
              [ 0.20, 0.20,-0.50],
              [ 1.00, 1.00, 1.00]])

# Printeamos A:
print(f'Matriz asociada al sistema lineal de ecuaciones: \n{A}')

Matriz asociada al sistema lineal de ecuaciones: 
[[-0.7   0.2   0.25]
 [ 0.5  -0.4   0.25]
 [ 0.2   0.2  -0.5 ]
 [ 1.    1.    1.  ]]


Cargamos el vector b:

In [8]:
# Vector B:
B = np.array([0, 0, 0, 1])

# Printeamos B:
print(f'Vector de términos independientes: \n{B}')

Vector de términos independientes: 
[0 0 0 1]


Dado que el solver de numpy solamente admite sistemas lineales cuadrados por el algoritmo que usa para la resolución [1], debemos eliminar una de las filas (cualquiera) de la matriz homogénea y quedarnos con la fila relacionada a la ecuación $ \sum_i{\pi_i} = 1$.
Hacemos lo mismo para el vector de términos independientes B.

Para hacer esto usamos la función el método delete de numpy, indicando la posición a eliminar y el eje (axis) al que pertenece.


[1] https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.linalg.solve.html

In [9]:
# Copio la matriz A original, para que no se modifique.
A_s = A.copy() 

# Eliminamos la primer fila de la matriz A:
A_s = np.delete(A_s, 0, 0)

# Printeamos:
print(f'Matriz asociada al sistema lineal de ecuaciones: \n{A_s}')
print(f'\n -> Dimensión: {A_s.shape}')

Matriz asociada al sistema lineal de ecuaciones: 
[[ 0.5  -0.4   0.25]
 [ 0.2   0.2  -0.5 ]
 [ 1.    1.    1.  ]]

 -> Dimensión: (3, 3)


In [10]:
# Copio el vector B original, para que no se modifique.
B_s = B.copy() 

# Eliminamos la primera componente del vector B:
B_s = np.delete(B_s, 0, 0)

print(f'\nVector de términos independientes: \n{B_s}')
print(f'\n -> Dimensión: {B_s.shape}')


Vector de términos independientes: 
[0 0 1]

 -> Dimensión: (3,)


Cumpliendo con un sistema cuadrado, usamos el método solve de numpy para obtener $x$ del sistema $Ax = B$

In [11]:
x = np.linalg.solve(A_s, B_s)
print('\n ** Vector solución de estado estable: \n %s' % x)


 ** Vector solución de estado estable: 
 [0.23809524 0.47619048 0.28571429]


### Forma alternativa 1: usando una matriz no cuadrada
Como explicamos anteriormente no podemos usar el método $solve$ en matrices no cuadradas. En su lugar podemos usar el método de los mínimos cuadrados para aproximar la solución[2]. Este método no tiene restricciones en cuanto a la dimensión de la matriz.

El desarrollo del método no forma parte de la materia, siendo contenido de Análisis Numérico y Cálculo Avanzado.

[2] https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html

In [12]:
x_lstsq, _, _, _ = np.linalg.lstsq(A, B, rcond=None)
print('\n ** Vector solución de estado estable: \n %s' % x_lstsq)


 ** Vector solución de estado estable: 
 [0.23809524 0.47619048 0.28571429]


### Forma alternativa 2: cargando directamente desde los datos

En la resolución original, usamos una matriz A relacionada al sistema lineal de ecuaciones que resolvimos a mano. Ahora veremos otra forma de llegar a la solución solamente con los datos dados y tratamiento de matrices.

Partiendo del sistema original: $\pi T = \pi$

Despejando $\pi$ obtenemos:

$(T^T - I) \pi^T = 0 $

Podemos transformar lo anterior en la notación que usamos más arriba para que tenga consistencia:

$A = (T^T - I)$

$X = \pi^T$

$B = 0$

Por lo tanto, llegamos a la misma expresión $Ax = B$

Entonces, comenzamos calculando: $A = (T^T - I)$

In [13]:
# Primero calculamos la traspuesta de la matriz de transición:
Tt = np.transpose(T)

print(f'\nT traspuesta: \n{Tt}')

# Luego con calculamos la matriz A, sabiendo que es la traspuesta de T menos la identidad.
A1 = Tt - np.identity(Tt.shape[0])

print(f'\nMatriz A: \n{A1}')


T traspuesta: 
[[0.3  0.2  0.25]
 [0.5  0.6  0.25]
 [0.2  0.2  0.5 ]]

Matriz A: 
[[-0.7   0.2   0.25]
 [ 0.5  -0.4   0.25]
 [ 0.2   0.2  -0.5 ]]


Seguimos con: $B = 0$

In [14]:
# El vector B, es un vector de ceros:
B1 = np.zeros(3)

print(f'\nVector B: \n{B1}')


Vector B: 
[0. 0. 0.]


A partir de aca, simplemente aplicamos el método que ya sabemos. Agregamos la información correspondiente a: $\sum_i{\pi_i} = 1$. 

In [15]:
# Copio la matriz A1 original, para que no se modifique.
A1_s = A1.copy() 

# Agregamos las probabilidades a la matriz A
eq_suma_p = np.array([[1, 1, 1]])

A1_s = np.concatenate((A1_s, eq_suma_p), axis=0)

# Printeamos:
print(f'Matriz A: \n{A1_s}')

Matriz A: 
[[-0.7   0.2   0.25]
 [ 0.5  -0.4   0.25]
 [ 0.2   0.2  -0.5 ]
 [ 1.    1.    1.  ]]


In [16]:
# Copio el vector B1 original, para que no se modifique.
B1_s = B1.copy() 

# Agregamos 1 al vector B:
B1_s = np.append(B1_s, 1)

# Printeamos:
print(f'\nVector B: \n{B1_s}')


Vector B: 
[0. 0. 0. 1.]


Resolvemos por mínimos cuadrados:

In [17]:
# Resolvemos con método de mínimos cuadrados:
x_lstsq, _, _, _ = np.linalg.lstsq(A1_s, B1_s, rcond=None)

# Printeamos la solucion:
print(f'\nVector solución de estado estable: {x_lstsq}')


Vector solución de estado estable: [0.23809524 0.47619048 0.28571429]


## Punto C: Clientes leales:

Los clientes leales los podemos encontrar en la diagonal de la matriz de transicion de tres pasos $T^3$. En este caso estamos considerando clientes que dejaron la marca en pasos anteriores pero que volvieron al 3er mes.

In [18]:
# Tercer punto: clientes leales a sus marcas en junio:
p_leal = np.diag(T_3)

print(f'Clientes que elegirían la marca nuevamente en junio: \n{p_leal}\n')

Clientes que elegirían la marca nuevamente en junio: 
[0.237 0.486 0.305]



Una alternativa interesante, es ver solamente quienes quedaron en cada estado sin migrar. Es decir considerar solamente como leales a los que atravesaron todos los meses sin cambiar. En este caso, recalculamos $T^3$ pero con la diagonal de T:

$T_{leal}^3 = [Diag(T)]^3$

In [19]:
# los clientes leales están ubicados sobre la diagonal de la matriz:
T_leal = np.diag(np.diag(T))

print(f'Matriz de transición de clientes leales: \n{T_leal}\n')
# Vamos a t=3 solamente con los leales:
T_leal_3 = np.linalg.matrix_power(T_leal, 3)

# Los clientes leales a t=3 son los ubicados sobre la diagonal:
prop_leales = np.dot(p_0, T_leal_3)
print(f'Distribución de clientes leales: \n{prop_leales}\n')

Matriz de transición de clientes leales: 
[[0.3 0.  0. ]
 [0.  0.6 0. ]
 [0.  0.  0.5]]

Distribución de clientes leales: 
[0.0054 0.0864 0.05  ]

